In [ ]:
# https://medium.com/codex/increasing-stock-returns-by-combining-williams-r-and-macd-in-python-5af999c90259

In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from math import floor
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

In [129]:
df = pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\old\data\\1d\XMRUSDT_1d.csv", index_col='Time')
df.index = pd.to_datetime(df.index)

In [130]:
# williams %R
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max()
    lowl = low.rolling(lookback).min()
    wr = -100 * ((highh - close) / (highh - lowl))
    return wr

df['wr_14'] = get_wr(df['High'], df['Low'], df['Close'], 14)



In [131]:
def get_macd(price, fast, slow, smooth):
    exp1 = price.ewm(span = fast, adjust=False).mean()
    exp2 = price.ewm(span = slow, adjust=False).mean()
    macd = pd.DataFrame(exp1-exp2).rename(columns={'Close':'macd'})
    
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    return macd, signal, hist
    
get_macd(df['Close'], 26, 12, 9)[1]

,signal
Time,
2019-08-03,0.000000
2019-08-04,0.004308
2019-08-05,-0.081296
2019-08-06,-0.160349
2019-08-07,-0.350922
...,...
2022-04-22,-13.301452
2022-04-23,-13.870136
2022-04-24,-14.059898


In [132]:
df['macd'] = get_macd(df['Close'], 26, 12, 9)[0]
df['macd_signal'] = get_macd(df['Close'], 26, 12, 9)[1]
df['macd_hist'] = get_macd(df['Close'], 26, 12, 9)[2]
df = df.dropna()

In [133]:
def implement_wr_macd_strategy(prices, wr, macd, macd_signal):    
    buy_price = []
    sell_price = []
    wr_macd_signal = []
    signal = 0
    # wr is df['wr_14']

    for i in range(len(wr)):
        if wr[i-1] > -50 and wr[i] < -50 and macd[i] > macd_signal[i]:
            # if the conditions are met, and im not holding a position, buy
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                wr_macd_signal.append(signal)

            # if the conditions are met, and im holding a position, stay put
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                wr_macd_signal.append(0)
                
        elif wr[i-1] < -50 and wr[i] > -50 and macd[i] < macd_signal[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                wr_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                wr_macd_signal.append(0)
        
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            wr_macd_signal.append(0)
            
    return buy_price, sell_price, wr_macd_signal
            
buy_price, sell_price, wr_macd_signal = implement_wr_macd_strategy(df['Close'], df['wr_14'], df['macd'], df['macd_signal'])


In [134]:
# len(df['Close']) == len(df['wr_14']) == len(df['macd']) == len(df['macd_signal']) == len(df['macd_hist'])
set(wr_macd_signal)

{-1, 0, 1}

In [135]:
# POSITION

position = []
# this for loop is only for pobulating the position list
# to be the same length as the wr_macd_signal list
# so we could add it to the same dataframe
for i in range(len(wr_macd_signal)):
    # maybe to make sure we dont have a nan in the list
    # or any other values than 1, -1, 0
    if wr_macd_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)
        
for i in range(len(df['Close'])):
    if wr_macd_signal[i] == 1:
        position[i] = 1
    elif wr_macd_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]
        
close_price = df['Close']
wr = df['wr_14']
macd_line = df['macd']
signal_line = df['macd_signal']
wr_macd_signal = pd.DataFrame(wr_macd_signal).rename(columns = {0:'wr_macd_signal'}).set_index(df.index)
position = pd.DataFrame(position).rename(columns = {0:'wr_macd_position'}).set_index(df.index)

frames = [close_price, wr, macd_line, signal_line, wr_macd_signal, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

strategy.head()

,Close,wr_14,macd,macd_signal,wr_macd_signal,wr_macd_position
Time,,,,,,
2019-08-16,82.25,-78.855721,1.181471,-0.081622,0,1
2019-08-17,81.52,-82.487562,1.527483,0.240199,0,1
2019-08-18,87.75,-51.492537,1.284188,0.448996,0,1
2019-08-19,89.32,-43.681592,0.953695,0.549936,0,1
2019-08-20,85.61,-62.139303,0.979848,0.635919,1,1


In [136]:
# BACKTESTING

df_ret = pd.DataFrame(np.diff(df['Close'])).rename(columns = {0:'returns'})
wr_macd_strategy_ret = []

for i in range(len(df_ret)):
    try:
        returns = df_ret['returns'][i] * strategy['wr_macd_position'][i]
        wr_macd_strategy_ret.append(returns)
    except:
        pass
    
wr_macd_strategy_ret_df = pd.DataFrame(wr_macd_strategy_ret).rename(columns = {0:'wr_macd_returns'})

investment_value = 100000
number_of_stocks = floor(investment_value / df['Close'][0])
wr_macd_investment_ret = []

for i in range(len(wr_macd_strategy_ret_df['wr_macd_returns'])):
    returns = number_of_stocks * wr_macd_strategy_ret_df['wr_macd_returns'][i]
    wr_macd_investment_ret.append(returns)

wr_macd_investment_ret_df = pd.DataFrame(wr_macd_investment_ret).rename(columns = {0:'investment_returns'})
total_investment_ret = round(sum(wr_macd_investment_ret_df['investment_returns']), 2)
profit_percentage = floor((total_investment_ret / investment_value) * 100)
print(cl('Profit gained from the W%R MACD strategy by investing $100k in df : {}'.format(total_investment_ret), attrs = ['bold']))
print(cl('Profit percentage of the W%R MACD strategy : {}%'.format(profit_percentage), attrs = ['bold']))

Profit gained from the W%R MACD strategy by investing $100k in df : 215735.4
Profit percentage of the W%R MACD strategy : 215%
